# The Battle of Neighborhoods


#### Data Science Capstone by IBM/Coursera 

           

## Table of Contents

* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction : Bussiness Problem  <a name="introduction"></a>

As people move from New York City to Toronto and vice versa, many are worried about the new changes they would bring in their lives. They are concerned about their neighborhoods and wish to move to a similar area they live in. It would be beneficial if they could know the neighborhood that matches their current one. This would ease their nervousness and help them properly focus on the actual neighborhood of their liking rather than regretting the choice after the move. Therefore we would try to identify and compare each area in New York and Toronto and create a similar grouping to help people identify their similar neighborhoods in the other city. 

We would plan to capture all the neighborhoods in New York and Toronto cities. Using the FourSquare integration, we could then find all types of places surrounding these neighborhoods. Break them into religious, restaurants, schools/colleges, parks, movie theaters, etc. Based on this information, we could then utilize K-means clustering to similar group neighborhoods together


## Data Section <a name="data"></a>

**Data Sources:** We use public libraries and API's in this project

1. **Public Data :**
 
    The New York data about their neighbourhoods would be downloaded from NYC Planning website. The data would list all the neighbourhoods and the geographic data. **[Link](https://www1.nyc.gov/site/planning/data-maps/open-data.page)**
                 
    The Toronto zip codes along with their neighbourhood names can be downloaded from the wiki page. **[Link](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M)**
                 
                 
2. **Foursquare API :** 

      Foursquare provides a valuable and publically accessible location information like the ameneties in nearby locations. We use their developer tools to access the required information about the neighboords in a city. Using these accessed information we then rank the neighborhoods based on the ameneties they have. This services is free of charge.


**K-Means Clustering Algorithm on the Data** 
                 
  We can use K-Means Clustering algorithm to group venues in a neighbourhood , then we can reduce the number of individual venue comparisons to be done against each neighborhood. We can do these comparisons against the types of venue, individually, collectively, or alltogether.
  
  

## Methodology <a name="methodology"></a>


**First Step** of the data analaysis is to pass the cordinates of neighbourhoods in New York and Toronto to the FourSquare and retriew the information all the venues about the 500 mile radius.   

**Second Step** would be to merge the information about the venues and its categories for all New York and Toronto into a single dataframe. 

**Third Step** would be to identify all the top 10 venues in each neighbourhood based on each venu's frequency.

**Forth Step** would be to run _k_-means to cluster the neighborhood into 5 clusters.

**Fifth Step** examine each cluster and determine the discriminating venue categories that distinguish each cluster.

**Sixth Step** is to provide an alternate if anyone wants to find the alternate of one neighbourhood in other city. This should also help if any one id planning to move withn the city.  



## ETL Data processing 

### Import libraries

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

from bs4 import BeautifulSoup

import json # library to handle JSON files

!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


! pip install folium==0.5.0
import folium # plotting library


import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

print('Libraries imported.')

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 79 kB 12.7 MB/s eta 0:00:01
  Created wheel for foli

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.datasets.samples_generator module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


### Understanding the New York Data 

In [2]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
print('Data downloaded!')

Data downloaded!


#### Exploring the New York Data

In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

#### Transform the data into a dataframe

In [4]:
# Extracting features from the data downloaded
neighborhoods_data = newyork_data['features']

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [5]:
## Retriving the geocordinates of New York City 

address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
ny_latitude = location.latitude
ny_longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(ny_latitude, ny_longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [6]:
# Creating a Map of New York for identifying the neighbourhoods 

# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[ny_latitude, ny_longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

### Defining Foursquare Credential and Version 


In [7]:
CLIENT_ID = 'IM5QHMFPY3VAOJFJDBAPL0RYWTLZ4HLWCHJ1BD3AXQG1M2MV' 
CLIENT_SECRET = 'DS0KEIQCBUZGBMU0UZNV2XWH1J2DUJ0RRLIUJCPX4ETT5SDR' 
ACCESS_TOKEN = 'RPONARM1LGWIQADKM5VQMATNKRHCNOEMIY1CUQC4R5L0R4S5' 
VERSION = '20180604'
LIMIT = 100 # A default Foursquare API limit value
radius = 500


#### Creating a Function to retrive different venues in different neighbourhood of New York 

In [8]:
def getNearbyVenues(names, borough, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, bor, lat, lng in zip(names, borough, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            bor,
            lat, 
            lng, 
            v['venue']['name'],
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'], 
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood',
                  'Neighborhood Borough',          
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',
                  'Venue Category']
    
    return(nearby_venues)

In [9]:
ny_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   borough=neighborhoods['Borough'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

**Checking the size of the data returned

In [10]:
print(ny_venues.shape)

ny_venues['City'] ='New York'

ny_venues.head()

(10199, 8)


,Neighborhood,Neighborhood Borough,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,City
0,Wakefield,Bronx,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop,New York
1,Wakefield,Bronx,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy,New York
2,Wakefield,Bronx,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop,New York
3,Wakefield,Bronx,40.894705,-73.847201,Walgreens,40.896528,-73.844700,Pharmacy,New York
4,Wakefield,Bronx,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop,New York


In [11]:
ny_venues.groupby(['Neighborhood','Neighborhood Borough','City']).count()

,,,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,Neighborhood Borough,City,,,,,,
Allerton,Bronx,New York,27,27,27,27,27,27
Annadale,Staten Island,New York,10,10,10,10,10,10
Arden Heights,Staten Island,New York,4,4,4,4,4,4
Arlington,Staten Island,New York,9,9,9,9,9,9
Arrochar,Staten Island,New York,19,19,19,19,19,19
Arverne,Queens,New York,21,21,21,21,21,21
Astoria,Queens,New York,100,100,100,100,100,100
Astoria Heights,Queens,New York,15,15,15,15,15,15
Auburndale,Queens,New York,19,19,19,19,19,19


## Understanding the Toronto Data 

In [12]:
wiki = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

soup = BeautifulSoup(wiki,"lxml")

table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df_Tor=pd.DataFrame(table_contents)
df_Tor['Borough']=df_Tor['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

df_Tor.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [13]:
!wget -O GeoSpataial_Data https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv

--2021-08-06 02:30:17--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2788 (2.7K) [text/csv]
Saving to: ‘GeoSpataial_Data’

GeoSpataial_Data    100%[===================>]   2.72K  --.-KB/s    in 0s      

2021-08-06 02:30:17 (77.0 MB/s) - ‘GeoSpataial_Data’ saved [2788/2788]



In [14]:
geospacial_data = pd.read_csv('GeoSpataial_Data')
geospacial_data = geospacial_data.rename(columns={'Postal Code': 'PostalCode'})
geospacial_data.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
df_Tor = df_Tor.join(geospacial_data.set_index('PostalCode'), on = 'PostalCode')

In [16]:
df_Tor.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [17]:
# Reset Index 
df_Tor = df_Tor.drop(['PostalCode'], axis=1)
df_Tor.head(10)

,Borough,Neighborhood,Latitude,Longitude
0,North York,Parkwoods,43.753259,-79.329656
1,North York,Victoria Village,43.725882,-79.315572
2,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,Etobicoke,Islington Avenue,43.667856,-79.532242
6,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,North York,Don Mills North,43.745906,-79.352188
8,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


#### Visualizing Toronto and its neighbourhoods 

In [18]:
## Retriving the geocordinates of Toronto City

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="explorer")
location = geolocator.geocode(address)
tor_latitude = location.latitude
tor_longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(tor_latitude, tor_longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


**Plotting the cordinates into the map

In [19]:
# Creating a Map of Toronto for identifying the neighbourhoods 

# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[tor_latitude, tor_longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_Tor['Latitude'], df_Tor['Longitude'], df_Tor['Borough'], df_Tor['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

#### Retrieve the venues from Foursquare for Toronto

In [20]:
tor_venues = getNearbyVenues(names=df_Tor['Neighborhood'],
                                   borough=df_Tor['Borough'],
                                   latitudes=df_Tor['Latitude'],
                                   longitudes=df_Tor['Longitude']
                                  )
tor_venues.shape

(2149, 8)

**Adding City to identify the city in the merged data. 

In [21]:
print(tor_venues.shape)

tor_venues['City'] ='Toronto'

tor_venues.head(20)

(2149, 8)


,Neighborhood,Neighborhood Borough,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,City
0,Parkwoods,North York,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park,Toronto
1,Parkwoods,North York,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant,Toronto
2,Parkwoods,North York,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop,Toronto
3,Victoria Village,North York,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena,Toronto
4,Victoria Village,North York,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop,Toronto
5,Victoria Village,North York,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant,Toronto
6,Victoria Village,North York,43.725882,-79.315572,Eglinton Ave E & Sloane Ave/Bermondsey Rd,43.726086,-79.313620,Intersection,Toronto
7,Victoria Village,North York,43.725882,-79.315572,Pizza Nova,43.725824,-79.312860,Pizza Place,Toronto
8,"Regent Park, Harbourfront",Downtown Toronto,43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery,Toronto
9,"Regent Park, Harbourfront",Downtown Toronto,43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop,Toronto


In [22]:
tor_venues.groupby(['Neighborhood','Neighborhood Borough','City']).count()

,,,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,Neighborhood Borough,City,,,,,,
Agincourt,Scarborough,Toronto,4,4,4,4,4,4
"Alderwood, Long Branch",Etobicoke,Toronto,8,8,8,8,8,8
"Bathurst Manor, Wilson Heights, Downsview North",North York,Toronto,23,23,23,23,23,23
Bayview Village,North York,Toronto,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",North York,Toronto,23,23,23,23,23,23
Berczy Park,Downtown Toronto,Toronto,58,58,58,58,58,58
"Birch Cliff, Cliffside West",Scarborough,Toronto,4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",West Toronto,Toronto,25,25,25,25,25,25
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",Downtown Toronto,Toronto,18,18,18,18,18,18


## Analysis <a name="analysis"></a>

Let start with a basic data Analysis task to find the deatils about the venue and its categories.  

#### Merging the data from New York and Toronto 

In [23]:
merge_data = pd.concat([ny_venues,tor_venues])
merge_data.head()

,Neighborhood,Neighborhood Borough,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,City
0,Wakefield,Bronx,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop,New York
1,Wakefield,Bronx,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy,New York
2,Wakefield,Bronx,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop,New York
3,Wakefield,Bronx,40.894705,-73.847201,Walgreens,40.896528,-73.844700,Pharmacy,New York
4,Wakefield,Bronx,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop,New York


In [24]:
merge_data.groupby(['Neighborhood','Neighborhood Borough','City']).count()

,,,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,Neighborhood Borough,City,,,,,,
Agincourt,Scarborough,Toronto,4,4,4,4,4,4
"Alderwood, Long Branch",Etobicoke,Toronto,8,8,8,8,8,8
Allerton,Bronx,New York,27,27,27,27,27,27
Annadale,Staten Island,New York,10,10,10,10,10,10
Arden Heights,Staten Island,New York,4,4,4,4,4,4
Arlington,Staten Island,New York,9,9,9,9,9,9
Arrochar,Staten Island,New York,19,19,19,19,19,19
Arverne,Queens,New York,21,21,21,21,21,21
Astoria,Queens,New York,100,100,100,100,100,100


### Analyze Each Neighborhood

In [25]:
# one hot encoding
venues_onehot = pd.get_dummies(merge_data[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
venues_onehot['Neighborhood'] = merge_data['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])
venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head()

,Yoga Studio,ATM,Accessories Store,Acupuncturist,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Auditorium,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Curling Ice,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Distribution Center,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Medical Supply Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Sta

In [26]:
venues_onehot.shape

(12348, 473)

In [27]:
venues_grouped = venues_onehot.groupby('Neighborhood').mean().reset_index()
venues_grouped.head()

,Neighborhood,Yoga Studio,ATM,Accessories Store,Acupuncturist,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Auditorium,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Curling Ice,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Distribution Center,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Medical Supply Store,Mediterranean Restaurant,Memorial Site,Men's St

#### Function to sort the venues in descending order to find the top 10

In [28]:

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

####  Identify the top 10 venues of the neighborhoods 

In [29]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = venues_grouped['Neighborhood']

for ind in np.arange(venues_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(venues_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Skating Rink,Breakfast Spot,Cultural Center,Cupcake Shop,Empanada Restaurant,English Restaurant,Entertainment Service,Escape Room
1,"Alderwood, Long Branch",Pizza Place,Pub,Playground,Sandwich Place,Pharmacy,Coffee Shop,Gym,Filipino Restaurant,Field,Dumpling Restaurant
2,Allerton,Pizza Place,Deli / Bodega,Cosmetics Shop,Chinese Restaurant,Discount Store,Supermarket,Breakfast Spot,Gas Station,Pharmacy,Bus Station
3,Annadale,Cosmetics Shop,Pizza Place,Diner,Pharmacy,Restaurant,American Restaurant,Train Station,Sushi Restaurant,Bar,Liquor Store
4,Arden Heights,Bus Stop,Pharmacy,Coffee Shop,Pizza Place,Falafel Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Escape Room


In [30]:
neighborhoods_venues_sorted.shape

(398, 11)

### Clustering Neighbourhoods to find similarities

#### Using silhouette score to find the optimum cluster count 


import random
from sklearn.metrics import silhouette_score

range_n_clusters = list (range(2,9))

venues_grouped_clustering = venues_grouped.drop('Neighborhood', 1)

random.seed(30)

for n_clusters in range_n_clusters:
    clusterer = KMeans(n_clusters=n_clusters)
    preds = clusterer.fit_predict(venues_grouped_clustering)
    centers = clusterer.cluster_centers_

    score = silhouette_score(venues_grouped_clustering, preds)
    print("For n_clusters = {}, silhouette score is {})".format(n_clusters, score))


Based on the silhouette method we could see that the score is max at n_clusters = 5

In [31]:
# set number of clusters
kclusters = 5

venues_grouped_clustering = venues_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(venues_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 0, 0, 4, 2, 2, 0, 4, 4, 0], dtype=int32)

#### Creating a dataframe with complete view based on our analysis

In [32]:
# Preping up the data for complete the missing information
final_df = merge_data.drop(['Venue','Venue Latitude','Venue Longitude','Venue Category'], axis=1)
final_df = final_df.drop_duplicates().reset_index(drop=True)
final_df.head()

,Neighborhood,Neighborhood Borough,Neighborhood Latitude,Neighborhood Longitude,City
0,Wakefield,Bronx,40.894705,-73.847201,New York
1,Co-op City,Bronx,40.874294,-73.829939,New York
2,Eastchester,Bronx,40.887556,-73.827806,New York
3,Fieldston,Bronx,40.895437,-73.905643,New York
4,Riverdale,Bronx,40.890834,-73.912585,New York


#### Merging the information with complete information to create a 

In [33]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# merge final_df with grouped data to add latitude/longitude for each neighborhood
final_df = final_df.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

final_df.head(10)

,Neighborhood,Neighborhood Borough,Neighborhood Latitude,Neighborhood Longitude,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wakefield,Bronx,40.894705,-73.847201,New York,0,Pharmacy,Sandwich Place,Caribbean Restaurant,Laundromat,Ice Cream Shop,Donut Shop,Dessert Shop,Deli / Bodega,Ethiopian Restaurant,Event Space
1,Co-op City,Bronx,40.874294,-73.829939,New York,0,Bus Station,Post Office,Restaurant,Fast Food Restaurant,Market,Bar,Grocery Store,Bagel Shop,Park,Donut Shop
2,Eastchester,Bronx,40.887556,-73.827806,New York,0,Caribbean Restaurant,Bus Station,Diner,Deli / Bodega,Seafood Restaurant,Pizza Place,Chinese Restaurant,Metro Station,Donut Shop,Bus Stop
3,Fieldston,Bronx,40.895437,-73.905643,New York,0,River,Bus Station,Athletics & Sports,Plaza,Yemeni Restaurant,Event Service,Eye Doctor,Exhibit,Event Space,Ethiopian Restaurant
4,Riverdale,Bronx,40.890834,-73.912585,New York,3,Park,Bus Station,Medical Supply Store,Gym,Bank,Baseball Field,Plaza,Playground,Exhibit,Event Space
5,Kingsbridge,Bronx,40.881687,-73.902818,New York,0,Pizza Place,Bar,Sandwich Place,Bakery,Latin American Restaurant,Mexican Restaurant,Supermarket,Burger Joint,Pharmacy,Fried Chicken Joint
6,Marble Hill,Manhattan,40.876551,-73.910660,New York,4,Sandwich Place,Gym,Coffee Shop,Department Store,Steakhouse,Supplement Shop,Optical Shop,Tennis Stadium,Seafood Restaurant,Donut Shop
7,Woodlawn,Bronx,40.898273,-73.867315,New York,0,Deli / Bodega,Pub,Pizza Place,Playground,Food Truck,Auto Workshop,Bar,Bus Station,Rental Car Location,Convenience Store
8,Norwood,Bronx,40.877224,-73.879391,New York,0,Pizza Place,Park,Chinese Restaurant,Bank,Pharmacy,Restaurant,American Restaurant,Mexican Restaurant,Caribbean Restaurant,Sandwich Place
9,Williamsbridge,Bronx,40.881039,-73.857446,New York,4,Soup Place,Nightclub,Bar,Spa,Falafel Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Escape Room


#### Agregate view on how the clusters are layed out 

In [34]:
aggr_view = pd.pivot_table(final_df,values = 'Neighborhood', index=['City','Neighborhood Borough'], columns=['Cluster Labels'],aggfunc='count')
aggr_view

Cluster Labels                      0    1     2    3     4
City     Neighborhood Borough                              
New York Bronx                   46.0  NaN   NaN  2.0   4.0
         Brooklyn                33.0  NaN   NaN  1.0  36.0
         Manhattan                2.0  NaN   NaN  NaN  38.0
         Queens                  49.0  NaN   3.0  4.0  25.0
         Staten Island           27.0  NaN  14.0  1.0  20.0
Toronto  Central Toronto          NaN  NaN   NaN  2.0   7.0
         Downtown Toronto         1.0  NaN   NaN  NaN  16.0
         Downtown Toronto Stn A   NaN  NaN   NaN  NaN   1.0
         East Toronto             NaN  NaN   NaN  NaN   4.0
         East Toronto Business    NaN  NaN   NaN  NaN   1.0
         East York                2.0  NaN   NaN  NaN   2.0
         East York/East Toronto   NaN  NaN   NaN  1.0   NaN
         Etobicoke                3.0  1.0   NaN  1.0   4.0
         Etobicoke Northwest      NaN  NaN   NaN  NaN   1.0
         Mississauga              NaN  NaN   NaN  NaN   1.0
         North York               4.0  1.0   1.0  5.0  12.0
         Queen's Park             NaN  NaN   NaN  NaN   1.0
         Scarborough              5.0  NaN   NaN  2.0   9.0
         West Toronto             NaN  NaN   NaN  NaN   6.0
         York                     1.0  NaN   NaN  1.0   2.0

#### Let's look at Cluster 3 as an example

In [35]:
Cluster_3 = final_df[final_df['Cluster Labels']==3]
Cluster_3

,Neighborhood,Neighborhood Borough,Neighborhood Latitude,Neighborhood Longitude,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Riverdale,Bronx,40.890834,-73.912585,New York,3,Park,Bus Station,Medical Supply Store,Gym,Bank,Baseball Field,Plaza,Playground,Exhibit,Event Space
27,Clason Point,Bronx,40.806551,-73.854144,New York,3,Park,South American Restaurant,Grocery Store,Convenience Store,Bus Stop,Moving Target,Pool,Boat or Ferry,Playground,Ethiopian Restaurant
91,Bergen Beach,Brooklyn,40.615150,-73.898556,New York,3,Harbor / Marina,Park,Moving Target,Athletics & Sports,Baseball Field,Playground,Factory,Empanada Restaurant,English Restaurant,Entertainment Service
148,South Ozone Park,Queens,40.668550,-73.809865,New York,3,Park,Deli / Bodega,Hotel,Bar,Donut Shop,Sandwich Place,Fast Food Restaurant,Food Truck,Fish Market,Fish & Chips Shop
188,Laurelton,Queens,40.667884,-73.740256,New York,3,Caribbean Restaurant,Cosmetics Shop,Park,Train Station,Duty-free Shop,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Escape Room
192,Somerville,Queens,40.597711,-73.796648,New York,3,Auto Workshop,Nature Preserve,Park,Surf Spot,Falafel Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Escape Room
203,Todt Hill,Staten Island,40.597069,-74.111329,New York,3,Park,Yemeni Restaurant,Falafel Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Event Service
302,Bayswater,Queens,40.611322,-73.765968,New York,3,Playground,Yemeni Restaurant,Falafel Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant
305,Parkwoods,North York,43.753259,-79.329656,Toronto,3,Food & Drink Shop,Park,Fast Food Restaurant,Yemeni Restaurant,Factory,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Escape Room
314,Glencairn,North York,43.709577,-79.445073,Toronto,3,Pizza Place,Park,Bakery,Japanese Restaurant,Yemeni Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Escape Room


## Results and Discussion  <a name="results"></a>

Based on our analysis, we found that New York has close to 10199 venues compared to Toronto, which had 2149 venues. Therefore, we can assume that New York is going to be bigger than Toronto. Consequently, we should find alternate neighborhoods that are very active and crowded; however, we might find it difficult to find remote and less populated spots.

New York has most of its neighborhoods spread around Cluster 0 and 4. Unfortunately, we could not define a similar location in cluster 1 for New York, and we did find that there are only two neighborhoods that qualified for Toronto.

Bronx and Brooklyn's neighborhoods dominate in Cluster 0, whereas Brooklyn and Manhattan neighborhoods dominate in Cluster 4. The majority of the downtown area of Toronto belongs to Cluster 4. This illustrates that anyone moving from the downtown area in either place should find it easier and comfortable in Manhattan or vice-versa.

Cluster 2 represents the prime locations of restaurants, so it should be suitable for people who would want to be somewhere in the middle but not very close to the action of the downtown.

Cluster 0 represents neighborhoods that are a bit far away from the business neighborhoods and hence rely on public transport for commuting.

Cluster 3 represents people who love parks or like physical activities; this is perhaps meant for young families with kids. Therefore they should find quite a selection in neighborhoods away from the busy and buzzing life of the City.

Cluster 4 represents restaurants and shopping neighborhoods. People who love to dine or shop would love these neighborhoods. 


## Conclusion <a name="conclusion"></a>

Therefore, based on our analysis, we are hopeful that this algorithm model would benefit people who are moving between two cities of New York and Toronto. We are optimistic that this would comfort people who are new to the city and want to find a similar environment as they were in their previous city. This should also be beneficial to those looking to move within the neighborhoods in the city as the model can also list similar neighborhoods in the same city. 

As we control the number of clusters, we did not increase the cluster count as it increases the risk of outliers for additional clusters, similar to what we see in Cluster 1. However, we could expand the number to fine-tune the similarities closer to its accuracy grouping.    

We were limited with the venue information provided by FourSquare, and adding additional information from other sources would help the K-means clustering model to be more accurate with new additional information. However, keeping in mind, the K-means is not great when dealing with the outliers as it would assign it to cluster based on the weakest similarities. Hence this model should be seen as helpful in shortlisting the neighborhoods. There should be additional research performed by the user to finalize and ensure the actual match. 